[구글 코랩(Colab)에서 실행하기](https://colab.research.google.com/github/lovedlim/bigdata_analyst_cert_v2/blob/main/part2/ch2/ch2_classification.ipynb)

# 1. 문제정의
- 도서 참고

# 2. 라이브러리 및 데이터 불러오기

In [ ]:
# 판다스 라이브러리
import pandas as pd

# 데이터 불러오기
# train = pd.read_csv('train.csv')
# test = pd.read_csv('test.csv')
train = pd.read_csv('https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch2/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch2/test.csv')

# EDA -> 전처리 -> 학습 -> 검증

In [78]:
# 레이블은 연소득이 $50000 이상과 미만으로 구분됨(컬럼명: income)
# 소득 예측값($50000 이상일 확률)을 csv파일로 생성
# 평가기준은 ROC-AUC로 평가 -> 분류다 그죠?
# 제출 파일은 예측값만 result.csv 파일로 생성해 제출(컬럼명: pred, 1개)

import pandas as pd

train = pd.read_csv('https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch2/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch2/test.csv')

# 1. EDA!!
# shape, sample, type, 기초통계량, 결측치, 이상치, 컬럼명
train.shape
# train.info()
# train.head()
# train.describe()
# test.describe()
# train.describe(include='O')
# train.isnull().sum() # age, workclass, occupation, hours.per.week	, native.country
# test.isnull().sum()

# target 별 빈도수
train['income'].value_counts() # 이게 양호?


,count
income,
<=50K,22263
>50K,7041


In [79]:
# 2. 전처리!!
# 결측치(필수)
# 이상치(선택)
# 인코딩(범주형시 필수)
# 스케일링(선택)

## 시계열데이터는 연도, 월, 일 파생변수 생성 추천

# 1. Missing value
# train = train.dropna()
# train.fillna(train)
# train.shape, train.shape # 대략 2000행 제거
# train.isnull().sum()
# train.isnull().sum()
# train = train
train['workclass'] = train['workclass'].fillna(train['workclass'].mode()[0])
train['native.country'] = train['native.country'].fillna(train['native.country'].mode()[0])
train['occupation'] = train['occupation'].fillna('X')
train.isnull().sum()

test['workclass'] = test['workclass'].fillna(train['workclass'].mode()[0])
test['native.country'] = test['native.country'].fillna(train['native.country'].mode()[0])
test['occupation'] = test['occupation'].fillna('X')
test.isnull().sum()

train['age'] = train['age'].fillna(int(train['age'].mean()))
train['hours.per.week'] = train['hours.per.week'].fillna(int(train['hours.per.week'].median()))
train.isnull().sum()


test['age'] = test['age'].fillna(int(train['age'].mean()))
test['hours.per.week'] = test['hours.per.week'].fillna(int(train['hours.per.week'].median()))
test.isnull().sum()




,0
id,0
age,0
workclass,0
fnlwgt,0
education,0
education.num,0
marital.status,0
occupation,0
relationship,0
race,0


In [80]:
# 2. Outliner
# 이상치 제거는 행 제거라서 train만 제거하고 test는 절 대 손 대 지 않는다.
# 고로 train만 drop

# train.describe()
print(train.shape)
train = train[train['age'] > 0]
print(train.shape)

(29304, 16)
(29301, 16)


In [95]:
# 3. Encoding
# 범주형 -> 수치형
# 	workclass	education	marital.status	occupation	relationship	race	sex	native.country	income
# train.describe(include='O')

# train은 label값을 포함하고 있고, label은 수치형 데이터가 아닌 범주형 데이터다. 따라서 인코딩을 진행하기에 앞서 함께 인코딩되는 것을 방지하기 위해 따로 담아둔다.
# train.pop('label')은 label을 대입하고 삭제하는 함수!
y_train = train.pop('income')

# # one-hot encoding = pd.get_dummies
# train_oh = pd.get_dummies(train)
# test_oh = pd.get_dummies(test)

# print(train.shape, test.shape, train_oh.shape, test_oh.shape)
# # (29301, 107) (3257, 103) train과 test의 컬럼 수 가 맞지 않다!!

from sklearn.preprocessing import LabelEncoder

cols = train.colums[train.dtype == 'object']

for col in cols:
  le = LabelEncoder()
  train[col] = le.fit_transform(train[col])
  test[col] = le.transform(test[col])

train.head()

,id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
0,3331,34.0,6,177331,15,10,2,9,0,2,1,4386,0,40.0,38
1,19749,58.0,3,290661,11,9,2,2,0,4,1,0,0,40.0,38
2,1157,48.0,3,125933,15,10,6,3,4,2,0,0,1669,38.0,38
3,693,58.0,3,100313,15,10,2,10,0,4,1,0,1902,40.0,38
4,12522,41.0,3,195661,15,10,2,13,0,4,1,0,0,54.0,38


In [94]:
# # 합치기 -> 원핫인코딩 -> 나누기
# data = pd.concat([train, test])
# data

# data_oh = pd.get_dummies(data)

# train_oh = data_oh.iloc[:len(train)]
# test_oh = data_oh.iloc[len(train):]

# train_oh.shape, test_oh.shape
# train_oh

,id,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,native.country_Scotland,native.country_South,native.country_Taiwan,native.country_Thailand,native.country_Trinadad&Tobago,native.country_United-States,native.country_Vietnam,native.country_Yugoslavia,income_<=50K,income_>50K
0,3331,34.0,177331,10,4386,0,40.0,False,False,False,...,False,False,False,False,False,True,False,False,False,True
1,19749,58.0,290661,9,0,0,40.0,False,False,False,...,False,False,False,False,False,True,False,False,True,False
2,1157,48.0,125933,10,0,1669,38.0,False,False,False,...,False,False,False,False,False,True,False,False,True,False
3,693,58.0,100313,10,0,1902,40.0,False,False,False,...,False,False,False,False,False,True,False,False,False,True
4,12522,41.0,195661,10,0,0,54.0,False,False,False,...,False,False,False,False,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29299,15999,28.0,47168,6,0,0,40.0,False,False,False,...,False,False,False,False,False,True,False,False,True,False
29300,21604,44.0,231793,16,0,0,38.0,False,True,False,...,False,False,False,False,False,True,False,False,True,False
29301,26839,41.0,201435,9,0,0,40.0,False,True,False,...,False,False,False,False,False,True,False,False,True,False
29302,16681,43.0,137722,9,0,0,40.0,False,False,False,...,False,False,False,False,False,True,False,False,True,False


In [ ]:
# 4. 스케일링
# 민맥스, 스탠더드, 로버스트

# 3. 탐색적 데이터 분석(EDA)

In [ ]:
# 데이터 샘플
train.head(3)

,id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,3331,34.0,State-gov,177331,Some-college,10,Married-civ-spouse,Prof-specialty,Husband,Black,Male,4386,0,40.0,United-States,>50K
1,19749,58.0,Private,290661,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40.0,United-States,<=50K
2,1157,48.0,Private,125933,Some-college,10,Widowed,Exec-managerial,Unmarried,Black,Female,0,1669,38.0,United-States,<=50K


In [ ]:
# 데이터 크기
train.shape, test.shape

((29304, 16), (3257, 15))

In [ ]:
# 자료형 (타입)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29304 entries, 0 to 29303
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              29304 non-null  int64  
 1   age             29292 non-null  float64
 2   workclass       27642 non-null  object 
 3   fnlwgt          29304 non-null  int64  
 4   education       29304 non-null  object 
 5   education.num   29304 non-null  int64  
 6   marital.status  29304 non-null  object 
 7   occupation      27636 non-null  object 
 8   relationship    29304 non-null  object 
 9   race            29304 non-null  object 
 10  sex             29304 non-null  object 
 11  capital.gain    29304 non-null  int64  
 12  capital.loss    29304 non-null  int64  
 13  hours.per.week  29291 non-null  float64
 14  native.country  28767 non-null  object 
 15  income          29304 non-null  object 
dtypes: float64(2), int64(5), object(9)
memory usage: 3.6+ MB


In [ ]:
# 수치형 컬럼 통계값 확인(train)
train.describe()

,id,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
count,29304.000000,29292.000000,2.930400e+04,29304.000000,29304.000000,29304.000000,29291.000000
mean,16264.027880,38.553223,1.897488e+05,10.080842,1093.858722,86.744506,40.434229
std,9384.518323,13.628811,1.055250e+05,2.570824,7477.435640,401.518928,12.324036
min,0.000000,-38.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,8145.750000,28.000000,1.177890e+05,9.000000,0.000000,0.000000,40.000000
50%,16253.500000,37.000000,1.783765e+05,10.000000,0.000000,0.000000,40.000000
75%,24374.250000,48.000000,2.370682e+05,12.000000,0.000000,0.000000,45.000000
max,32560.000000,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [ ]:
# 수치형 컬럼 통계값 확인(test)
test.describe()

,id,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
count,3257.000000,3251.000000,3.257000e+03,3257.000000,3257.000000,3257.000000,3248.000000
mean,16423.704943,38.802830,1.900447e+05,10.079214,931.804728,92.336199,40.468288
std,9535.416746,13.917588,1.057902e+05,2.590118,6496.962999,415.732721,12.598546
min,3.000000,17.000000,1.882700e+04,1.000000,0.000000,0.000000,1.000000
25%,8078.000000,28.000000,1.186520e+05,9.000000,0.000000,0.000000,40.000000
50%,16626.000000,37.000000,1.783190e+05,10.000000,0.000000,0.000000,40.000000
75%,24743.000000,48.000000,2.364360e+05,12.000000,0.000000,0.000000,45.000000
max,32559.000000,90.000000,1.033222e+06,16.000000,99999.000000,3900.000000,99.000000


In [ ]:
# 범주형 컬럼 통계값 확인 (train)
train.describe(include='O')

,workclass,education,marital.status,occupation,relationship,race,sex,native.country,income
count,27642,29304,29304,27636,29304,29304,29304,28767,29304
unique,8,16,7,14,6,5,2,41,2
top,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States,<=50K
freq,20451,9449,13466,3683,11845,25022,19578,26240,22263


In [ ]:
# 범주형 컬럼 통계값 확인 (test)
test.describe(include='object')

,workclass,education,marital.status,occupation,relationship,race,sex,native.country
count,3083,3257,3257,3082,3257,3257,3257,3211
unique,8,16,7,14,6,5,2,37
top,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States
freq,2245,1052,1510,457,1348,2794,2212,2930


In [ ]:
# 결측치 (train)
train.isnull().sum()

,0
id,0
age,12
workclass,1662
fnlwgt,0
education,0
education.num,0
marital.status,0
occupation,1668
relationship,0
race,0


In [ ]:
# 결측치 (test)
test.isnull().sum()

,0
id,0
age,6
workclass,174
fnlwgt,0
education,0
education.num,0
marital.status,0
occupation,175
relationship,0
race,0


In [ ]:
# label(target)별 개수 확인
train['income'].value_counts()

,count
income,
<=50K,22263
>50K,7041


# 4. 데이터 전처리

1. 결측치 처리 (결측치가 있다면 필수)
2. 이상치 처리 (선택)
3. 인코딩 (범주형 데이터가 있다면 필수)
4. 스케일링 (선택)

## 1. 결측치 처리
- 범주형
- 수치형

### 결측치 삭제

In [ ]:
# 결측치가 있는 데이터(행) 전체 삭제 및 확인 dropna() / 기본값 axis=0
print("처리전:", train.shape)
df = train.dropna()
print("처리후:", df.shape)

처리전: (29304, 16)
처리후: (27096, 16)


In [ ]:
# 결측치가 있는 특정컬럼 데이터(행)삭제
df = train.dropna(subset=['native.country','workclass'])
df.isnull().sum()

,0
id,0
age,12
workclass,0
fnlwgt,0
education,0
education.num,0
marital.status,0
occupation,6
relationship,0
race,0


In [ ]:
# 결측치가 있는 컬럼 삭제 dropna(axis=1)
print("처리전:", train.shape)
df = train.dropna(axis=1)
print("처리후:", df.shape)

처리전: (29304, 16)
처리후: (29304, 11)


In [ ]:
# 결측치가 많은 특정 컬럼 삭제 drop
print("처리전:", train.shape)
df = train.drop(['native.country','workclass'], axis=1)
print("처리후:", df.shape)

처리전: (29304, 16)
처리후: (29304, 14)


### 결측치 채우기 (범주형)

In [ ]:
# 최빈값
m = train['workclass'].mode()[0]
train['workclass'] = train['workclass'].fillna(m)

m = train['native.country'].mode()[0]
train['native.country'] = train['native.country'].fillna(m)

train.isnull().sum()

,0
id,0
age,12
workclass,0
fnlwgt,0
education,0
education.num,0
marital.status,0
occupation,1668
relationship,0
race,0


In [ ]:
# 결측값을 새로운 카테고리로 생성 X_train['occupation']
train['occupation'] = train['occupation'].fillna('X')
train.isnull().sum()

,0
id,0
age,12
workclass,0
fnlwgt,0
education,0
education.num,0
marital.status,0
occupation,0
relationship,0
race,0


In [ ]:
# test데이터
test['workclass'] = test['workclass'].fillna(train['workclass'].mode()[0])
test['native.country']  = test['native.country'].fillna(train['native.country'].mode()[0])
test['occupation'] = test['occupation'].fillna("X")

### 결측치 채우기 (수치형)

In [ ]:
# age 컬럼 평균값으로 채우기
value = int(train['age'].mean())
print("평균값:", value)
train['age'] = train['age'].fillna(value)
test['age'] = test['age'].fillna(value)

평균값: 38


In [ ]:
# 주당 근무시간 중앙값으로 채우기
value = int(train['hours.per.week'].median())
print("중앙값:", value)
train['hours.per.week'] = train['hours.per.week'].fillna(value)
test['hours.per.week'] = test['hours.per.week'].fillna(value)

중앙값: 40


In [ ]:
train.isnull().sum()

,0
id,0
age,0
workclass,0
fnlwgt,0
education,0
education.num,0
marital.status,0
occupation,0
relationship,0
race,0


## 이상치 처리

In [ ]:
train.describe()

,id,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
count,29304.000000,29304.000000,2.930400e+04,29304.000000,29304.000000,29304.000000,29304.000000
mean,16264.027880,38.552996,1.897488e+05,10.080842,1093.858722,86.744506,40.434036
std,9384.518323,13.626025,1.055250e+05,2.570824,7477.435640,401.518928,12.321306
min,0.000000,-38.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,8145.750000,28.000000,1.177890e+05,9.000000,0.000000,0.000000,40.000000
50%,16253.500000,37.000000,1.783765e+05,10.000000,0.000000,0.000000,40.000000
75%,24374.250000,48.000000,2.370682e+05,12.000000,0.000000,0.000000,45.000000
max,32560.000000,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [ ]:
train[train['age']<=0]

,id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
39,29188,-33.0,Private,263561,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,60.0,United-States,>50K
79,14325,-38.0,Private,22245,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,60.0,United-States,>50K
26161,4292,-25.0,Private,200681,Some-college,10,Never-married,X,Own-child,White,Male,0,0,40.0,United-States,<=50K


In [ ]:
test[test['age']<=0]

,id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country


In [ ]:
# age가 1이상인 데이터만 살림
print(train.shape)
train = train[train['age']>0]
print(train.shape)

(29304, 16)
(29301, 16)


전처리 주의사항
- 이상치, 결측치에 대해 test 데이터(행) 삭제 불가
    - 예) test 데이터 100개가 주어지고 100개로 평가를 하는데 임의로 10개를 삭제해버리면 채점을 할 수가 없음
    - train 데이터는 학습/훈련용 데이터임. 데이터가 많을 경우 임의로 소수 데이터(행) 삭제해도 무방함
- test, train 컬럼은 삭제/추가 가능. 단, train과 컬럼수와 명이 일치해야 함 (y(target) 제외)


## 인코딩

In [ ]:
y_train = train.pop("income")

### 원핫(one-hot) 인코딩

In [ ]:
train_oh = pd.get_dummies(train)
test_oh = pd.get_dummies(test)
print(train.shape, test.shape, train_oh.shape, test_oh.shape)

(29301, 15) (3257, 15) (29301, 107) (3257, 103)


In [ ]:
print(train.shape, test.shape)
data = pd.concat([train, test], axis=0)
data_oh = pd.get_dummies(data)
train_oh = data_oh.iloc[:len(train)].copy()
test_oh = data_oh.iloc[len(train):].copy()
print(train_oh.shape, test_oh.shape)

(29301, 15) (3257, 15)
(29301, 107) (3257, 107)


### 레이블(label) 인코딩

In [ ]:
# cols = train.select_dtypes(include='object').columns #방법1
# cols = train.columns[train.dtypes == object] #방법2
cols = ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex','native.country']
cols

['workclass',
 'education',
 'marital.status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native.country']

In [ ]:
from sklearn.preprocessing import LabelEncoder

for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

train.head()

,id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
0,3331,34.0,6,177331,15,10,2,9,0,2,1,4386,0,40.0,38
1,19749,58.0,3,290661,11,9,2,2,0,4,1,0,0,40.0,38
2,1157,48.0,3,125933,15,10,6,3,4,2,0,0,1669,38.0,38
3,693,58.0,3,100313,15,10,2,10,0,4,1,0,1902,40.0,38
4,12522,41.0,3,195661,15,10,2,13,0,4,1,0,0,54.0,38


In [ ]:
# from sklearn.preprocessing import LabelEncoder

# train_le = train.copy()
# test_le = test.copy()

# for col in cols:
#     le = LabelEncoder()
#     train_le[col] = le.fit_transform(train[col])
#     test_le[col] = le.transform(test[col])

# train_le.head()

## 스케일링

In [ ]:
# 수치형 컬럼명
cols = ['age', 'fnlwgt', 'education.num', 'capital.gain', 'capital.loss', 'hours.per.week']

In [ ]:
# 데이터를 매번 새롭게 불러오기 위해 함수로 제작 함
def get_data():
    train_copy = train.copy()
    test_copy = test.copy()
    return train_copy, test_copy

In [ ]:
# 민-맥스 스케일링 MinMaxScaler (모든 값이 0과 1사이)
train_copy, test_copy = get_data()

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
display(train_copy[cols].head(2))
train_copy[cols] = scaler.fit_transform(train_copy[cols])
test_copy[cols] = scaler.transform(test_copy[cols])
display(train_copy[cols].head(2))

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
0,34.0,177331,10,4386,0,40.0
1,58.0,290661,9,0,0,40.0


,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
0,0.232877,0.112092,0.600000,0.04386,0.0,0.397959
1,0.561644,0.189060,0.533333,0.00000,0.0,0.397959


In [ ]:
# 표준화 StandardScaler (Z-score 정규화, 평균이 0 표준편차가 1인 표준 정규분포로 변경)
train_copy, test_copy = get_data()

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
display(train_copy[cols].head(2))
train_copy[cols] = scaler.fit_transform(train_copy[cols])
test_copy[cols] = scaler.transform(test_copy[cols])
display(train_copy[cols].head(2))

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
0,34.0,177331,10,4386,0,40.0
1,58.0,290661,9,0,0,40.0


,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
0,-0.335121,-0.117705,-0.031462,0.440247,-0.216056,-0.035121
1,1.428590,0.956277,-0.420430,-0.146298,-0.216056,-0.035121


In [ ]:
# 로버스트 스케일링 : 중앙값과 사분위 값 활용, 이상치 영향 최소화 장점
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
display(train[cols].head(2))
train[cols] = scaler.fit_transform(train[cols])
test[cols] = scaler.transform(test[cols])
display(train[cols].head(2))

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
0,34.0,177331,10,4386,0,40.0
1,58.0,290661,9,0,0,40.0


,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
0,-0.15,-0.008711,0.000000,4386.0,0.0,0.0
1,1.05,0.941438,-0.333333,0.0,0.0,0.0


## target(label)

In [ ]:
y_train.value_counts()

,count
income,
<=50K,22262
>50K,7039


In [ ]:
# target = y_train.map({'<=50K':0, '>50K':1})
# target.value_counts()

In [ ]:
# target = y_train.replace('<=50K', 0).replace('>50K', 1)
# target.value_counts()

# 5. 검증 데이터 분할

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train,
                                                  y_train,
                                                  test_size=0.2,
                                                  random_state=0)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((23440, 15), (5861, 15), (23440,), (5861,))

In [ ]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)
pred=rf.predict_proba(X_val) # 각 레이블에 속할 확률 값 반환

print(rf.classes_)
pred[:10]

['<=50K' '>50K']


array([[1.  , 0.  ],
       [1.  , 0.  ],
       [0.9 , 0.1 ],
       [0.63, 0.37],
       [1.  , 0.  ],
       [0.99, 0.01],
       [0.98, 0.02],
       [0.94, 0.06],
       [0.12, 0.88],
       [0.88, 0.12]])

# 6. 머신러닝 학습 및 평가


In [ ]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)
pred=rf.predict_proba(X_val) # 각 레이블에 속할 확률 값 반환
pred[:10]

array([[1.  , 0.  ],
       [1.  , 0.  ],
       [0.9 , 0.1 ],
       [0.63, 0.37],
       [1.  , 0.  ],
       [0.99, 0.01],
       [0.98, 0.02],
       [0.94, 0.06],
       [0.12, 0.88],
       [0.88, 0.12]])

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(y_val, pred[:,1])
print('roc_auc:', roc_auc)

roc_auc: 0.9173623004487484


In [ ]:
pred = rf.predict(X_val) # 예측된 각 레이블 반환
pred[:10]

array(['<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K', '<=50K',
       '<=50K', '>50K', '<=50K'], dtype=object)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_val, pred)
print('accuracy_score:', accuracy)

accuracy_score: 0.8694761986009213


In [ ]:
from sklearn.metrics import f1_score
f1 = f1_score(y_val, pred, pos_label='>50K')
print('f1_score:', f1)

f1_score: 0.6926476496584973


In [ ]:
# LightGBM
import lightgbm as lgb
lgbmc = lgb.LGBMClassifier(random_state=0, verbose=-1)
lgbmc.fit(X_train, y_train)
pred = lgbmc.predict_proba(X_val)

roc_auc = roc_auc_score(y_val, pred[:,1])
print('roc_auc:', roc_auc)

pred = lgbmc.predict(X_val)
accuracy = accuracy_score(y_val, pred)
print('accuracy_score:', accuracy)

f1 = f1_score(y_val, pred, pos_label='>50K')
print('f1_score:', f1)

roc_auc: 0.9279535666686397
accuracy_score: 0.8771540692714553
f1_score: 0.7158642462509865


# 7. 예측 및 결과 파일 생성

In [ ]:
pred = lgbmc.predict_proba(test)
pred

array([[0.89924007, 0.10075993],
       [0.97622077, 0.02377923],
       [0.9853122 , 0.0146878 ],
       ...,
       [0.93528667, 0.06471333],
       [0.98996872, 0.01003128],
       [0.96723292, 0.03276708]])

In [ ]:
lgbmc.classes_

array(['<=50K', '>50K'], dtype=object)

In [ ]:
submit = pd.DataFrame({'pred':pred[:,1]})
submit.to_csv("result.csv", index=False)

In [ ]:
pd.read_csv("result.csv")

,pred
0,0.100760
1,0.023779
2,0.014688
3,0.798048
4,0.026542
...,...
3252,0.009613
3253,0.321992
3254,0.064713
3255,0.010031
